In [22]:
import pandas as pd
import numpy as np
from pprint import pprint
import sqlite3

In [23]:
summary_df = pd.read_csv("summary_df.csv", index_col=0, parse_dates=True)
ranking_df = pd.read_csv("ranking_df.csv", index_col=0, parse_dates=True)
views_df = pd.read_csv("views_df.csv", index_col=0, parse_dates=True)
earnings_df = pd.read_csv("earnings_df.csv", index_col=0, parse_dates=True)

In [24]:
combined_df = pd.merge(summary_df, ranking_df, on='name')
combined_df = pd.merge(combined_df, views_df, on='name')
combined_df = pd.merge(combined_df, earnings_df, on='name')

In [25]:
combined_df.count()

category_x              1315
created                 1626
name                    1626
subscribers             1626
uploads                 1626
views                   1626
category_y              1315
grade                   1626
socialblade_rank        1626
subscriber_rank         1626
view_rank               1626
category_x              1315
subs_last30d_count      1626
views_last30d_count     1626
category_y              1315
max_annual_earnings     1626
max_monthly_earnings    1626
min_annual_earnings     1626
min_monthly_earnings    1626
dtype: int64

In [26]:
cols=pd.Series(combined_df.columns)
for dup in combined_df.columns.get_duplicates(): cols[combined_df.columns.get_loc(dup)]=[dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(combined_df.columns.get_loc(dup).sum())]
combined_df.columns=cols
combined_df = combined_df[["category_x.1", "created", "name", "subscribers", "uploads", "views", "grade", "socialblade_rank", "subscriber_rank", "view_rank", "subs_last30d_count", "views_last30d_count","min_monthly_earnings", "max_monthly_earnings", "min_annual_earnings", "max_annual_earnings"]]
combined_df = combined_df.dropna().rename(columns={"category_x.1":"category"})
combined_df = combined_df.rename(columns={"min_monthly_earnings":"monthly_earnings"})

C:\Users\uulka\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'get_duplicates' is deprecated and will be removed in a future release. You can use idx[idx.duplicated()].unique() instead
  


In [27]:
combined_df = combined_df[combined_df['subscribers'] >= 0]
combined_df = combined_df[combined_df['views'] > 0]
combined_df = combined_df[combined_df['subs_last30d_count'] > 0]
combined_df = combined_df[combined_df['views_last30d_count'] > 0]
combined_df = combined_df[combined_df['monthly_earnings'] > 0]

In [28]:
combined_df.count()

category                1121
created                 1121
name                    1121
subscribers             1121
uploads                 1121
views                   1121
grade                   1121
socialblade_rank        1121
subscriber_rank         1121
view_rank               1121
subs_last30d_count      1121
views_last30d_count     1121
monthly_earnings        1121
max_monthly_earnings    1121
min_annual_earnings     1121
max_annual_earnings     1121
dtype: int64

In [29]:
summary = combined_df[['category', 'monthly_earnings', 'subscribers', 'views', 'uploads']].groupby(['category']).mean().astype(np.int64)
summary

,monthly_earnings,subscribers,views,uploads
category,,,,
Animals,11120,2141071,738694619,696
Autos,6455,1358424,383536914,661
Comedy,13639,5008017,1297376517,1199
Education,20026,3377660,1589255902,488
Entertainment,13509,2975862,1279749914,4776
Film,11943,2200461,1035872918,908
Games,11325,4437635,1746559285,1995
Howto,15150,5729361,1225525979,731
Music,15473,3993768,1983268280,1763


In [30]:
summary.to_csv('summary.csv')

In [31]:
conn = sqlite3.connect('YouTube.sqlite')

In [32]:
summary.to_sql('summary', conn, if_exists='replace', index=True)

In [33]:
pd.read_sql('select * from summary', conn)

,category,monthly_earnings,subscribers,views,uploads
0,Animals,11120,2141071,738694619,696
1,Autos,6455,1358424,383536914,661
2,Comedy,13639,5008017,1297376517,1199
3,Education,20026,3377660,1589255902,488
4,Entertainment,13509,2975862,1279749914,4776
5,Film,11943,2200461,1035872918,908
6,Games,11325,4437635,1746559285,1995
7,Howto,15150,5729361,1225525979,731
8,Music,15473,3993768,1983268280,1763
9,News,13528,3004897,1169301152,40607
